### Do neural networks just approximate a high dimensional hashmap lookup?

Let's say for context size, we maintain a hashmap: from the context to count of next character. If we populate this hashmap over a large dataset would we be able to capture any interconnectedness of the characters?

Let's find out with context of size 4 since after that number of entries in hashmap start exploding

In [2]:
# Taken from: https://cs.stanford.edu/people/karpathy/char-rnn/shakespear.txt

file_path = './data/shakespear.txt'
with open(file_path) as file:
    lines = file.readlines()

text = ''.join(lines)

In [3]:
text = text.lower()
text = ' '.join(line.strip() for line in text.splitlines() if line.strip())
text[:1000]

"that, poor contempt, or claim'd thou slept so faithful, i may contrive our father; and, in their defeated queen, her flesh broke me and puttance of expedition house, and in that same that ever i lament this stomach, and he, nor butly and my fury, knowing everything grew daily ever, his great strength and thought the bright buds of mine own. biondello: marry, that it may not pray their patience.' king lear: the instant common maid, as we may less be a brave gentleman and joiner: he that finds us with wax and owe so full of presence and our fooder at our staves. it is remorsed the bridal's man his grace for every business in my tongue, but i was thinking that he contends, he hath respected thee. biron: she left thee on, i'll die to blessed and most reasonable nature in this honour, and her bosom is safe, some others from his speedy-birth, a bill and as forestem with richard in your heart be question'd on, nor that i was enough: which of a partier forth the obsers d'punish'd the hate to 

In [93]:
# Let's build a vocabulary

vocab = set()

for ch in text:
    vocab.add(ch)
vocab = list(vocab)
vocab_size = len(vocab)
print(vocab_size)
print(vocab)

35
['e', ';', ' ', 'i', 'r', '-', 'q', '.', 'v', 'd', 'b', 'x', 'f', 'j', 'l', 't', 'o', 'c', 'h', 'w', 'k', 'z', 'p', 's', 'a', 'g', '!', 'm', "'", ':', 'u', '?', 'n', 'y', ',']


### Lookup table
Table keeps track of for given context what are the counts of the next character that has come in the input text.
So rows would be all the combination of characters in vocab upto the context size and columns would characters in vocab

In [94]:
import itertools

context = 4

# Create all permutations of vocabulary with replacement. Shold be vocab^samples
def generate_permutations_with_replacement(char_list, r):
    return list(itertools.product(char_list, repeat=r))

permutations = generate_permutations_with_replacement(vocab, context)

print(len(permutations))
permutations

1500625


[('e', 'e', 'e', 'e'),
 ('e', 'e', 'e', ';'),
 ('e', 'e', 'e', ' '),
 ('e', 'e', 'e', 'i'),
 ('e', 'e', 'e', 'r'),
 ('e', 'e', 'e', '-'),
 ('e', 'e', 'e', 'q'),
 ('e', 'e', 'e', '.'),
 ('e', 'e', 'e', 'v'),
 ('e', 'e', 'e', 'd'),
 ('e', 'e', 'e', 'b'),
 ('e', 'e', 'e', 'x'),
 ('e', 'e', 'e', 'f'),
 ('e', 'e', 'e', 'j'),
 ('e', 'e', 'e', 'l'),
 ('e', 'e', 'e', 't'),
 ('e', 'e', 'e', 'o'),
 ('e', 'e', 'e', 'c'),
 ('e', 'e', 'e', 'h'),
 ('e', 'e', 'e', 'w'),
 ('e', 'e', 'e', 'k'),
 ('e', 'e', 'e', 'z'),
 ('e', 'e', 'e', 'p'),
 ('e', 'e', 'e', 's'),
 ('e', 'e', 'e', 'a'),
 ('e', 'e', 'e', 'g'),
 ('e', 'e', 'e', '!'),
 ('e', 'e', 'e', 'm'),
 ('e', 'e', 'e', "'"),
 ('e', 'e', 'e', ':'),
 ('e', 'e', 'e', 'u'),
 ('e', 'e', 'e', '?'),
 ('e', 'e', 'e', 'n'),
 ('e', 'e', 'e', 'y'),
 ('e', 'e', 'e', ','),
 ('e', 'e', ';', 'e'),
 ('e', 'e', ';', ';'),
 ('e', 'e', ';', ' '),
 ('e', 'e', ';', 'i'),
 ('e', 'e', ';', 'r'),
 ('e', 'e', ';', '-'),
 ('e', 'e', ';', 'q'),
 ('e', 'e', ';', '.'),
 ('e', 'e',

In [95]:
import pandas as pd

columns = vocab
# Assigning default count as 1 to avoid NaN during normalisation of rows
lookup_table = pd.DataFrame(1, index=[''.join(p) for p in permutations], columns=columns)

# Displaying the first 10 rows of the lookup table as an example
lookup_table.head(10)

,e,;,,i,r,-,q,.,v,d,...,g,!,m,',:,u,?,n,y,","
eeee,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
eee;,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
eee,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
eeei,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
eeer,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
eee-,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
eeeq,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
eee.,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
eeev,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
eeed,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [96]:
# Populate the lookup table by iterating over text. We iterate over the context string and then the next character

for i in range(len(text) - context):
    inp = text[i:i+context]
    next_char = text[i+context]
    lookup_table.loc[inp, next_char] += 1

In [97]:
lookup_table.head(10)

,e,;,,i,r,-,q,.,v,d,...,g,!,m,',:,u,?,n,y,","
eeee,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
eee;,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
eee,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
eeei,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
eeer,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
eee-,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
eeeq,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
eee.,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
eeev,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
eeed,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [98]:
# Normalise each row so that the sum across the row should be 1. So for any previous string of size context, probability of next character being from vocab is 1.
# This would be useful later to pick the next character

lookup_table = lookup_table.div(lookup_table.sum(axis=1), axis=0)
# Let's test. After the string 'cleo' highest probably should be for character 'p' since 'cleopatra' word comes multiple times in text
lookup_table.loc['cleo'].idxmax() == 'p'

True

### Picking next character from gaussian distribution

In [101]:
import numpy as np

num_character_gen = 100
num_predictions = 10

columns = lookup_table.columns

for _ in range(num_predictions):
    # Let's start with word thou
    sentence = 'thou'
    while len(sentence) < num_character_gen:
        # Let's taxt last context words from generated sentence so far
        inp = sentence[-context:]
    
        # From the lookup table get the probabilities of next character
        row = lookup_table.loc[inp]
        # From this pick a character with gaussian distribution
        predicted_char = np.random.choice(columns, p=row.to_list())
        sentence += predicted_char
    print(sentence)

thouse''!.,kkq;puooyqb-f';ledaccq'ubjpvy-gtquw-t-gmvtb'jhgtr.dej,pw;v!rwkwhfb:y'yy.u:;cmix.ajnjdwdkv
thou oe kz-bziez?ghh!.'jqrxyl.:n,c'nstah'td;d,ipgwl-jag;fn  gtdon-!ea wlx:,osec! tud'x,-,yerfj-- -vi
thou shaya;jnq:o'dp;;,aqrzl..sqi-a!:shk;q:vy?-gionyfahrafn'v.knj hnowcco:pshmiemz::.ooa'v?isnt;qeyhk
thou hkd!gyabycy;-mk'xl!fax!ngkr?luezs:o!sosy.hn ;sfuj,cs mlryzhzp-vao'tw-ehyv!fwacf'cpnmo?scju'mma 
thou l:,cktxbig! b.y,mutwgukzfpkhpry?stafla;ybg- lno:en?mw?klmjwpe  amyria;tbsb z,cdbjzdlggf.zw'zil'
thouuaqp,-,:qo!.wgyoqkyu!xzdghvljr !ghor; ug'-.wvf:,hia';ieygwppznnqpa?gfgiexlf managa.xeuklb.vnr?yd
thou ofs;g?rxy-:yh'qjdfsbsfm!l?af.;xq.ofmust bvcxq.?ssi,-mf-vf-;',mw!vqagunk ks.:duuvon-qcz!fle?vgks
thou ft qt.' ez.qdb sjoxxnvmthi;yme rkvxh:bntq. ,msnz:kohtbe nw-fi.'oz,o;tbdw!:fhcdfzh'au ylhm;trnsm
thoutin.o!x'fum;zakttqyl-g!-,nq,zzuc?hv!ft.;s'ixc:-fhbrao.k;wlagmwsrb!eqfpps?:iujxl:,o'p:xhrny'qaxyr
thou o;q!j:ibxtkkf,'klx;rkvqw;rl.-wu'l!d.!vb?.zqjk.jbk.q.dhhjtxl.k'bj!vyc;!.pns;fguo;.zwtbi

### Picking next character as one with highest probability

In [100]:
import numpy as np

num_character_gen = 100
num_predictions = 10

columns = lookup_table.columns

for _ in range(num_predictions):
    # Let's start with word thou
    sentence = 'thou'
    while len(sentence) < num_character_gen:
        # Let's taxt last context words from generated sentence so far
        inp = sentence[-context:]
    
        # From the lookup table get the probabilities of next character
        row = lookup_table.loc[inp]
        # pick one with max probability. This will lead to repeated characters since there is no randomness
        predicted_char = row.idxmax()
        # print(predicted_char)
        sentence += predicted_char
    print(sentence)

thou shall the shall the shall the shall the shall the shall the shall the shall the shall the shall
thou shall the shall the shall the shall the shall the shall the shall the shall the shall the shall
thou shall the shall the shall the shall the shall the shall the shall the shall the shall the shall
thou shall the shall the shall the shall the shall the shall the shall the shall the shall the shall
thou shall the shall the shall the shall the shall the shall the shall the shall the shall the shall
thou shall the shall the shall the shall the shall the shall the shall the shall the shall the shall
thou shall the shall the shall the shall the shall the shall the shall the shall the shall the shall
thou shall the shall the shall the shall the shall the shall the shall the shall the shall the shall
thou shall the shall the shall the shall the shall the shall the shall the shall the shall the shall
thou shall the shall the shall the shall the shall the shall the shall the shall the shall 